In [2]:
!pip install stable-baselines3[extra]
!pip install tensorflow
%load_ext tensorboard
import tensorflow as tf
import datetime
from stable_baselines3 import DQN

  Using cached Shimmy-0.2.1-py3-none-any.whl.metadata (2.3 kB)
  Using cached AutoROM-0.6.1-py3-none-any.whl.metadata (2.4 kB)
  Using cached AutoROM.accept-rom-license-0.6.1.tar.gz (434 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
INFO: pip is looking at multiple versions of shimmy[atari] to determine which version is compatible with other requirements. This could take a while.
   ---------------------------------------- 0.0/952.1 kB ? eta -:--:--
   --------------------------------------- 952.1/952.1 kB 22.3 MB/s eta 0:00:00
  Attempting uninstall: gymnasium
    Found existing installation: gymnasium 0.28.1
    Uninstalling gymnasium-0.28.1:
      Successfully uninstalled gymnasium-0.28.1
  Attemptin

In [4]:
DQN?

Init signature:
DQN(
    policy: Union[str, type[stable_baselines3.dqn.policies.DQNPolicy]],
    env: Union[gymnasium.core.Env, ForwardRef('VecEnv'), str],
    learning_rate: Union[float, Callable[[float], float]] = 0.0001,
    buffer_size: int = 1000000,
    learning_starts: int = 100,
    batch_size: int = 32,
    tau: float = 1.0,
    gamma: float = 0.99,
    train_freq: Union[int, tuple[int, str]] = 4,
    gradient_steps: int = 1,
    replay_buffer_class: Optional[type[stable_baselines3.common.buffers.ReplayBuffer]] = None,
    replay_buffer_kwargs: Optional[dict[str, Any]] = None,
    optimize_memory_usage: bool = False,
    n_steps: int = 1,
    target_update_interval: int = 10000,
    exploration_fraction: float = 0.1,
    exploration_initial_eps: float = 1.0,
    exploration_final_eps: float = 0.05,
    max_grad_norm: float = 10,
    stats_window_size: int = 100,
    tensorboard_log: Optional[str] = None,
    policy_kwargs: Optional[dict[str, Any]] = None,
    verbose: int = 0,

try:
  !rm -rf boptestGymService
except:
  pass
!git clone -b boptest-gym-service https://github.com/ibpsa/project1-boptest-gym.git boptestGymService

In [6]:
import requests
import sys
import torch
import gymnasium as gym
import os

In [8]:
sys.path.insert(0,'src/boptestGymService')

url = 'http://localhost:80'

In [10]:
from stable_baselines3 import SAC
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold
from boptestGymService.boptestGymEnv import BoptestGymEnv
from boptestGymEnv import DiscretizedActionWrapper
from gym.wrappers import FlattenObservation
from boptestGymEnv import DiscretizedObservationWrapper

In [12]:
# Redefine reward function
class BoptestGymEnvCustomReward(BoptestGymEnv):
    def get_reward(self):
        '''Custom reward function. We use the BOPTEST `GET /kpis` API call to compute the
        total cummulative discomfort from the beginning of the episode. Note
        that this is the true value that BOPTEST uses when evaluating
        controllers.

        'tdis_tot': temp discomfort
         'idis_tot': 0,
         'ener_tot': total energy
         'cost_tot': total cost
         'emis_tot': total emission
         'pele_tot': defines the HVAC peak electrical demand.
         'pgas_tot': defines the HVAC peak gas demand.
         'pdih_tot': defines the HVAC peak district heating demand.
         'time_rat': defines the average ratio between the controller computation time and the test simulation control step. The controller computation time is measured as the time between two emulator advances
        '''
        # Compute BOPTEST core kpis
        kpis = requests.get('{0}/kpi/{1}'.format(self.url, self.testid)).json()['payload']
        # Calculate objective integrand function as the total discomfort
        objective_integrand = kpis['tdis_tot']
        # Give reward if there is not immediate increment in discomfort
        if objective_integrand == self.objective_integrand:
          reward=1
        else:
          reward=0
        # Record current objective integrand for next evaluation
        self.objective_integrand = objective_integrand
        return reward

In [14]:
test_case="multizone_office_simple_air"

In [16]:
BoptestGymEnv?

Init signature:
BoptestGymEnv(
    url='http://127.0.0.1',
    testcase='bestest_hydronic_heat_pump',
    actions=['oveHeaPumY_u'],
    observations={'reaTZon_y': (280.0, 310.0)},
    reward=['reward'],
    max_episode_length=10800,
    random_start_time=False,
    excluding_periods=None,
    regressive_period=None,
    predictive_period=None,
    start_time=0,
    warmup_period=0,
    scenario={'electricity_price': 'constant'},
    step_period=900,
    render_episodes=False,
    log_dir='C:\\Users\\irmak\\Documents\\GitHub\\RL_ex',
)
Docstring:     
BOPTEST Environment that follows gym interface.
This environment allows the interaction of RL agents with building
emulator models from BOPTEST. 
 
Init docstring:
Parameters
----------
url: string
    Rest API url for communication with the BOPTEST interface
testcase: string
    The string identifier of the testcase
actions: list
    List of strings indicating the action space. The bounds of 
    each variable from the action space the ar

In [18]:
lower_setp = 21 + 273.15
upper_setp = 24 + 273.15

env = BoptestGymEnvCustomReward(url                   = url,
                    testcase              = test_case,
                    actions               = ['hvac_oveZonSupCor_TZonCooSet_u', 
                                             'hvac_oveZonSupNor_TZonCooSet_u',
                                             'hvac_oveZonSupSou_TZonCooSet_u'],
                    observations          = {"hvac_reaZonCor_TZon_y":(lower_setp, upper_setp),
                                             "hvac_reaZonNor_TZon_y":(lower_setp, upper_setp),
                                             "hvac_reaZonSou_TZon_y":(lower_setp, upper_setp)},
                    random_start_time     = False,
                    start_time            = 154*24*3600,
                    max_episode_length    = 24*3600,
                    warmup_period         = 24*3600,
                    step_period           = 900,
                    render_episodes= False)

C:\Users\irmak\anaconda3\Lib\site-packages\gymnasium\spaces\box.py:235: UserWarning: WARN: Box low's precision lowered by casting to float32, current low.dtype=float64
  gym.logger.warn(
C:\Users\irmak\anaconda3\Lib\site-packages\gymnasium\spaces\box.py:305: UserWarning: WARN: Box high's precision lowered by casting to float32, current high.dtype=float64
  gym.logger.warn(


In [10]:
log_path = os.path.join( "local_files", "Logs")

In [11]:
model = SAC('MlpPolicy', env, verbose=1, learning_rate=0.0003, gamma=0.99, batch_size=256, tensorboard_log= log_path)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [12]:
model.learn(total_timesteps=10)

Logging to src\local_files\Logs\SAC_9


evaluate_policy(model, env, n_eval_episodes=3)

In [38]:
%tensorboard --logdir ./src/local_files/Logs/SAC_9/

Reusing TensorBoard on port 6007 (pid 29012), started 0:01:20 ago. (Use '!kill 29012' to kill it.)

In [20]:
env.stop()

In [ ]:
from stable_baselines3 import SAC
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold

In [ ]:
    env = md.create_env(url, test_case, True)
    model = SAC('MlpPolicy', env, verbose=1, learning_rate=0.0003, gamma=0.99, batch_size=256,
                tensorboard_log=log_path)
    SAC.load("SAC_model", env = env)

    done = False
    obs, _ = env.reset()

    from IPython.display import clear_output
    while not done:
        # Clear the display output at each step
        #clear_output(wait=True)
        # Compute control signal
        action = model.predict(obs, deterministic=True)
        # Print the current operative temperature and decided action
        print('-------------------------------------------------------------------')
        print('State  [Bin #]  = {:.0f}'.format(obs))
        print('Action [ - ]    = {:.0f}'.format(action))
        print('-------------------------------------------------------------------')
        # Implement action
        obs, reward, terminated, truncated, info = env.step(action)  # send the action to the environment
        done = (terminated or truncated)

In [ ]:
    env = md.create_env(url, test_case, True)
    model = SAC('MlpPolicy', env, verbose=1, learning_rate=0.0003, gamma=0.99, batch_size=256,
                tensorboard_log=log_path)
    SAC.load("SAC_model", env = env)

    done = False
    obs, _ = env.reset()

    from IPython.display import clear_output
    while not done:
        # Clear the display output at each step
        #clear_output(wait=True)
        # Compute control signal
        action = model.predict(obs, deterministic=True)
        # Print the current operative temperature and decided action
        print('-------------------------------------------------------------------')
        print('State  [Bin #]  = {:.0f}'.format(obs))
        print('Action [ - ]    = {:.0f}'.format(action))
        print('-------------------------------------------------------------------')
        # Implement action
        obs, reward, terminated, truncated, info = env.step(action)  # send the action to the environment
        done = (terminated or truncated)